In [ ]:
# 2. Mount Drive & Paths
import os, sys

if "google.colab" in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT = "/content/drive/MyDrive/Colab Notebooks/headway-prediction"
else:
    ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

sys.path.insert(0, ROOT)
print(f"Root: {ROOT}")

In [ ]:
# 3. Imports
from src.config import Config
from src.data.dataset import SubwayDataGenerator
from src.models.convlstm import ConvLSTM
from src.training.trainer import Trainer
from src.evaluator import Evaluator
print("✅ Imports complete")

In [ ]:
# 4. Load, Split, Scale, Generate
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

config = Config()
config.DATA_DIR = os.path.join(ROOT, "data")

# Load
data_gen = SubwayDataGenerator(config)
data_gen.load_data(normalize=False, load_temporal=False)

# Split indices
T = len(data_gen.headway_data)
train_end = int(T * config.TRAIN_SPLIT)
val_end = int(T * (config.TRAIN_SPLIT + config.VAL_SPLIT))

# Scale (fit on train only)
scaler = MinMaxScaler()
scaler.fit(data_gen.headway_data[:train_end].reshape(-1, 1))
data_gen.headway_data = scaler.transform(data_gen.headway_data.reshape(-1, 1)).reshape(data_gen.headway_data.shape)
data_gen.schedule_data = scaler.transform(data_gen.schedule_data.reshape(-1, 1)).reshape(data_gen.schedule_data.shape)

# Save scaler
os.makedirs(os.path.join(ROOT, "models"), exist_ok=True)
joblib.dump(scaler, os.path.join(ROOT, "models", "scaler.pkl"))

# Generate datasets
train_ds = data_gen.make_dataset(0, train_end, shuffle=True, include_temporal=False)
val_ds = data_gen.make_dataset(train_end, val_end, shuffle=False, include_temporal=False)
test_ds = data_gen.make_dataset(val_end, T, shuffle=False, include_temporal=False)

print(f"Train: 0-{train_end}, Val: {train_end}-{val_end}, Test: {val_end}-{T}")

In [ ]:
# 5. Config & Build Model
print(f"Batch: {config.BATCH_SIZE}, LR: {config.LEARNING_RATE}, Kernel: {config.KERNEL_SIZE}")

model_builder = ConvLSTM(config, use_temporal=False)
model = model_builder.build_model()
model.summary()

In [ ]:
# 6. Train
trainer = Trainer(model, config, checkpoint_dir=os.path.join(ROOT, "models"))
trainer.compile_model()

history = trainer.fit(train_ds, val_ds, patience=config.EARLY_STOPPING_PATIENCE)

In [ ]:
# 7. Evaluate
evaluator = Evaluator(config, scaler=scaler)

# Test metrics
test_loss, test_rmse, test_r2 = model.evaluate(test_ds, verbose=0)
print(f"\nTest RMSE: {test_rmse:.1f}s, R²: {test_r2:.4f}")

# Training curves
save_dir = os.path.join(ROOT, "images")
os.makedirs(save_dir, exist_ok=True)
evaluator.plot_training_curves(history, save_path=os.path.join(save_dir, "training_curves.png"))

# Predictions
evaluator.plot_spatiotemporal_prediction(model, test_ds, sample_idx=0, direction=0,
    save_path=os.path.join(save_dir, "prediction_north.png"))